# Detecting and Replacing Outliers

In [ ]:
import pandas as pd
import numpy as np
import os 

In [ ]:
filename = os.path.join(os.getcwd(), "data", "adult.data.partial")
df = pd.read_csv(filename, header=0)

###  Get the Dimensions of the Dataset

In [ ]:
df.shape

### Glance at the Data

In [ ]:
df.head()

## Step 1: Compute the n-th Percentile of a Given Column

As an analyst, your goal is to detect the outliers in the `hours-per-week` column. In particular, you want to get the 99.9th percentile of the values in the `hours-per-week` column.<br>

As was discussed in the videos, *z-scores* can be used to compute the n-th percentile of a data array. Toward the end of this notebook, we will be looking at a few ways to compute the z-scores and then figure out the n-th percentile in a data column. For now, however, we will show you a ready-made method from `numpy` that achieves our objective.

The code cell below uses the `np.percentile()` function and gets the value of `hours-per-week` that corresponds to the 99.9th percentile.

In [ ]:
hpw_999 = np.percentile(df['hours-per-week'], 99.9)
hpw_999

In the code cell below, figure out the value of `education-num` that corresponds to the 90th percentile of the education in years. Hint: Use the same method as the code cell above, but replace the column name and the percentage value. Save your results to variable `edu_90`.

### Graded Cell
The cell below will be graded. Remove the line "raise NotImplementedError()" before writing your code.

In [ ]:
edu_90 = np.percentile(df['education-num'], 90.0)
edu_90

### Self-Check

Run the cell below to test the correctness of your code above before submitting for grading. Do not add code or delete code in the cell.

In [ ]:
# Run this self-test cell to check your code; 
# do not add code or delete code in this cell
from jn import testEdu

try:
    p, err = testEdu(df, edu_90)
    print(err)
except Exception as e:
    print("Error!\n" + str(e))
    


## Step 2: Add a Column With the Windsorized Version of the Original Column.

In this next section, we will use a new package called SciPy, which stands for Scientific Python. For more information about SciPy, consult the online [documentation](https://scipy.github.io/devdocs/reference/index.html).

First, import the `stats` module from the `scipy` package.

In [ ]:
import scipy.stats as stats

Read the documentation for the function `stats.mstats.winsorize()`.

In [ ]:
stats.mstats.winsorize?

This function will create a copy of a given column, such that the outlier values will be replaced. In particular, you will pass two percentage cutoffs as a list to the `limits` parameter, and all the column values below the specified lower percentile cutoff, as well as all the values above the upper cutoff, will be replaced with the corresponding percentile value. 

The code cell below uses the `stats.mstats.windsorize()` function to add a new column to DataFrame `df`. The column will be named `education-num-win` and will contain the windsorized version of the `education-num` column, with the cutoff from the 'bottom' and the cutoff from the 'top' both set at the 1% level.

In [ ]:
df['education-num-win'] = stats.mstats.winsorize(df['education-num'], limits=[0.01, 0.01])
df.head(15)

## Deep Dive: Computing z-scores 

First, let's review what the *z-score of a given value* is. <br>
Say your dataset contains a feature (aka a one-dimensional array, a vector, a list, a variable, a data column) called $X$, and you want to compute the z-score for one particular observation (aka an example value, a cell) of this feature. Let's call this observation $x_i$. <br>
A z-score of $x_i$ is given by:
$$z = \frac{x_i-\bar{X}}{s},$$
where $\bar{X}$ is the mean of all the values of $x$ in your data, and $s$ is the standard deviation of those values.<br>

The code cells below implements this formula.

#### Calculate a z-score for one (given) value, a given mean, and a given standard deviation

In [ ]:
F_mean = 5.44
F_std = 7.7
value = 4 

value_zscore = (value-F_mean)/F_std
value_zscore

####  Calculate a z-score for one (given) value, given the full sample of values. (The `numpy` way)

In [ ]:
F = [4, 6, 3, -3, 4, 5, 6, 7, 3 , 8, 1, 9, 1, 2, 2, 35, 4, 1]
value = F[0]

F_std = np.std(F)
F_mean = np.mean(F)
value_zscore = (value-F_mean)/F_std
value_zscore

In [ ]:
F_mean

#### Calculate the z-score for all values of a feature vector. (The `numpy` way)

All we need to do now is to apply the computation we implemented above to every value in the feature vector `F`. 

In [ ]:
F_std = np.std(F)
F_mean = np.mean(F)
zscores = []
for value in F:
    z = (value-F_mean)/F_std
    zscores.append(z)
    
zscores

Now, let's write code that implements the same computation the *pythonic* way -- using *list comprehensions*. <br>
Tip: remember that list comprehension syntax looks like this: <br>
`[action_to_apply(new_var_name) for new_var_name in list_containing_values]`

In [ ]:
F_std = np.std(F)
F_mean = np.mean(F)
zscores = [(value-F_mean)/F_std for value in F]
zscores

#### Calculate the z-score for all values of a feature vector. (The `scipy` way)

Previously we were computing the z-score by implementing its definition formula via `numpy`.<br>
This time, we will use a ready-made function `zscore()` from the package `scipy`.

In [ ]:
zscores = stats.zscore(df['hours-per-week'])
zscores

####  Calculate z-scores for all values of all (numeric) columns

We will demonstrate how to use the Pandas `apply()` method to broadcast the same function (`stats.zscore`) onto all columns in a (filtered!) DataFrame:

In [ ]:
df_zscores = df.select_dtypes(include=['number']).apply(stats.zscore)
df_zscores.head(10)